# **Algoritmos geneticos para optimizacion combinatoria**
Flores Estopier Rodrigo

Escribe un algoritmo genetico que permita resolver el problema de TSP para 10 ciudades.
Por ejemplo, un tour valido seria [2,4,6,1,3,5,7,10,8,9]
Las distancias entre cada una de las ciudades

In [13]:



import random
from IPython import display as display

from functools import cmp_to_key


import numpy
M=2*numpy.ones([11,11])
M[1][3]=1; M[3][5]=1; M[5][7]=1; M[7][9]=1;
M[9][2]=1; M[2][4]=1; M[4][6]=1; M[6][8]=1;
M[8][10]=1

#Imprime la matriz con indices
print("Matriz de distancias")
#Imprime los indices de arriba
print(" ",end="  ")
for i in range(11):
  print(i,end="  ")
print()
#Imprime los indices de la izquierda
for i in range(11):
  print(i,M[i])

L_chromosome= len(M[0])-1

#Numero de cromosoams
N_chromosomes=150
#probability of mutation
prob_m=0.1

#Poblacion inicial
F0=[]
fitness_values=[]

def random_chromosome():
  global L_chromosome
  chromosome = list(range(1, L_chromosome + 1))
  random.shuffle(chromosome)
  return chromosome



#Inicializar poblacion inicial
for i in range(0,N_chromosomes):
    F0.append(random_chromosome())
    fitness_values.append(0)

#Funcion de aptitud
def fun_aptitud(chromosome):
    distance = 0
    for i in range(len(chromosome) - 1):
        distance += M[chromosome[i]][chromosome[i + 1]]
    distance += M[chromosome[-1]][chromosome[0]]  # Regreso a la ciudad inicial
    return distance


def evaluate_chromosomes():
    global F0
    for p in range(N_chromosomes):
        fitness_values[p]=fun_aptitud(F0[p])


def compare_chromosomes(chromosome1,chromosome2):
    fvc1=fun_aptitud(chromosome1)
    fvc2=fun_aptitud(chromosome2)
    if fvc1 > fvc2:
        return 1
    elif fvc1 == fvc2:
        return 0
    else: #fvg1<fvg2
        return -1


Lwheel=N_chromosomes*10

def create_wheel():
    global F0,fitness_values

    maxv=max(fitness_values)
    acc=0
    for p in range(N_chromosomes):
        acc+=maxv-fitness_values[p]
    fraction=[]
    for p in range(N_chromosomes):
        fraction.append( float(maxv-fitness_values[p])/acc)
        if fraction[-1]<=1.0/Lwheel:
            fraction[-1]=1.0/Lwheel
##    print fraction
    fraction[0]-=(sum(fraction)-1.0)/2
    fraction[1]-=(sum(fraction)-1.0)/2
##    print fraction

    wheel=[]

    pc=0

    for f in fraction:
        Np=int(f*Lwheel)
        for i in range(Np):
            wheel.append(pc)
        pc+=1

    return wheel

#Inicializamos la siguiente generacion
F1=F0[:]
n=0
# indexes=list(range(N_chromosomes))

# Ordered crossover (OX)
def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [None] * len(parent1)
    child[start:end] = parent1[start:end]

    pointer = end
    for gene in parent2:
        if gene not in child:
            if pointer >= len(parent2):
                pointer = 0
            child[pointer] = gene
            pointer += 1
    return child


  # Exchange mutation
def mutate(chromosome):
    a, b = random.sample(range(len(chromosome)), 2)
    chromosome[a], chromosome[b] = chromosome[b], chromosome[a]

def nextgeneration(b):
    global n
    #display.clear_output(wait=True)
    #display.display(button)
    F0.sort(key=cmp_to_key(compare_chromosomes) )
    #print( "Best solution so far:")
    n+=1
    print( n," ",F0[0],"= ", fun_aptitud(F0[0]) )

    #elitism, the two best chromosomes go directly to the next generation
    F1[0]=F0[0]
    F1[1]=F0[1]
    fitness=[]

    roulette=create_wheel()
    #print (roulette)
    for i in range(0,int((N_chromosomes-2)/2)):
        #Two parents are selected
        p1=random.choice(roulette)
        p2=random.choice(roulette)
        #Two descendants are generated
        o1=crossover(F0[p1],F0[p2])
        o2=crossover(F0[p2],F0[p1])
        #Each descendant is mutated with probability prob_m
        #print(o1)
        if random.random() < prob_m:
            mutate(o1)

        #print("Mutacion? ",o1)
        if random.random() < prob_m:
            mutate(o2)
        #The descendants are added to F1
        F1[2+2*i]=o1
        F1[3+2*i]=o2


    #The generation replaces the old one
    F0[:]=F1[:]






evaluate_chromosomes()
F0.sort(  key=cmp_to_key(compare_chromosomes))




Matriz de distancias
   0  1  2  3  4  5  6  7  8  9  10  
0 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
1 [2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2.]
2 [2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2.]
3 [2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2.]
4 [2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2.]
5 [2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2.]
6 [2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2.]
7 [2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2.]
8 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1.]
9 [2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2.]
10 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [14]:
#Realizar distintas iteraciones
for i in range(500):
  nextgeneration(0)



1   [1, 3, 10, 8, 9, 2, 4, 5, 7, 6] =  16.0
2   [1, 3, 10, 8, 9, 2, 4, 5, 7, 6] =  16.0
3   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
4   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
5   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
6   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
7   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
8   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
9   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
10   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
11   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
12   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
13   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
14   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
15   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
16   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
17   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
18   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
19   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
20   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
21   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
22   [6, 1, 3, 5, 8, 7, 9, 2, 10, 4] =  15.0
23   [6, 1, 3, 5, 8